In [57]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

#问题3模型

In [58]:
# 读取Excel数据
data = pd.read_excel('no_error.xlsx')

# 定义自变量X和因变量y
X = data[['母亲年龄','教育程度','CBTS', 'EPDS', 'HADS']]#'EPDS',
#X = data[['母亲年龄','CBTS', 'EPDS', 'HADS']]
y = data['婴儿行为特征']
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)
mean=np.mean(X,axis=0)
X1=X-mean
std=np.std(X1,axis=0)
X_scaled=X1/std
# 添加截距项
poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(X_scaled)
# 建立多元回归模型
model = sm.OLS(y, X_poly).fit()
# 输出模型摘要
print(model.summary())
#预测部分
data1 = pd.read_excel('pred.xlsx')
# 定义自变量X和因变量y
X_test = data1[['母亲年龄','教育程度','CBTS',  'EPDS','HADS']]#
X_test=X_test-mean
X_test=X_test/std
X_test_poly = poly.fit_transform(X_test)
# 预测
y_pred = model.predict(X_test_poly)
print(y_pred)
# 输出模型摘要
#print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                 婴儿行为特征   R-squared (uncentered):                   0.800
Model:                            OLS   Adj. R-squared (uncentered):              0.766
Method:                 Least Squares   F-statistic:                              23.46
Date:                Sun, 06 Aug 2023   Prob (F-statistic):                    5.94e-84
Time:                        13:13:50   Log-Likelihood:                         -477.01
No. Observations:                 378   AIC:                                      1064.
Df Residuals:                     323   BIC:                                      1280.
Df Model:                          55                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#参数

In [59]:
k1 = (2812-200)/3
k2 = (1890-500)/2
k3 = (12500-300)/5
b1 = 200
b2 = 500
b3 = 300

#问题四模型

In [60]:
# 读取数据
data = pd.read_excel("no_error1.xlsx")

# 提取睡眠质量指标并进行归一化
sleep_quality = data[["整晚睡眠时间", "睡醒次数", "入睡方式"]]
scaler = MinMaxScaler()
scaled_sleep_quality = scaler.fit_transform(sleep_quality)

In [61]:
# 熵权法计算指标权重
def entropy_weight(data):
    # 为每个指标的取值范围加上微小值
    data = data + 0.001
    # 计算熵
    p = data / data.sum()
    log_p = np.log(p)
    e = -(p * log_p).sum()
    # 计算权重
    w = (1 - e) / (3 - e)
    return w

In [62]:
weights0 = entropy_weight(scaled_sleep_quality[:,0])
weights1 = entropy_weight(scaled_sleep_quality[:,1])
weights2 = entropy_weight(scaled_sleep_quality[:,2])

In [63]:
# 计算综合睡眠质量得分
# 睡眠时间越长越好
scaled_sleep_time_score = weights1 * scaled_sleep_quality[:, 0]
# 入睡方式越大越好
scaled_sleep_way_score = weights0 * scaled_sleep_quality[:, 2]
# 睡醒次数越少越好
scaled_wakeup_score = weights2 * (1 - scaled_sleep_quality[:, 1])

In [64]:
# 计算相似度得分
similarity_score = scaled_sleep_time_score + scaled_sleep_way_score + scaled_wakeup_score

# 归一化相似度得分
normalized_similarity_score = (similarity_score - similarity_score.min()) / (similarity_score.max() - similarity_score.min())

# 添加综合睡眠质量列
data["综合睡眠质量"] = pd.cut(normalized_similarity_score, bins=[-0.001, 0.45, 0.60, 0.75, 1], labels=["差", "中", "良", "优"])

# 输出结果
print(data[["编号", "综合睡眠质量"]])

      编号 综合睡眠质量
0      1      中
1      2      优
2      3      良
3      4      中
4      5      优
..   ...    ...
374  387      优
375  388      中
376  389      差
377  390      优
378  238      中

[379 rows x 2 columns]


In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# 读取数据
data1 = pd.read_excel("no_error1.xlsx")

# 提取特征和标签
X = data1[["母亲年龄", "婚姻状况", "教育程度", "妊娠时间", "分娩方式", "CBTS", "EPDS", "HADS"]]
mean2=np.mean(X,axis=0)
X=X-mean2
std2=np.std(X,axis=0)
X=X/std2
#X = data1[["母亲年龄","教育程度", "分娩方式", "CBTS", "EPDS", "HADS"]]
y=normalized_similarity_score*100
#y = data["综合睡眠质量"]

# 划分训练集和测试集
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
data2 = pd.read_excel("pred.xlsx")
X1=data2[["母亲年龄", "婚姻状况", "教育程度", "妊娠时间", "分娩方式", "CBTS", "EPDS", "HADS"]]
X1=X1-mean2
X1=X1/std2

In [76]:
#多项式回归
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
#X = sm.add_constant(X)
poly2 = PolynomialFeatures(degree=4, include_bias=False)
X_poly2 = poly2.fit_transform(X)
# 建立多元回归模型
rf_model = sm.OLS(y, X_poly).fit()
#X1=sm.add_constant(X1)
X1_poly2 = poly.fit_transform(X1)
# 输出模型摘要
#print(model.summary())
y_pred=rf_model.predict(X1_poly2)
show= pd.cut(y_pred/100, bins=[-0.001, 0.45, 0.60, 0.75, 100], labels=["差", "中", "良", "优"])
show=show.T
type(show)



pandas.core.arrays.categorical.Categorical

In [113]:
b = [[24,2,2,26.5,1,15,22,18]]
columns = ["母亲年龄", "婚姻状况", "教育程度", "妊娠时间", "分娩方式", "CBTS", "EPDS", "HADS"]
df = pd.DataFrame(b, columns=columns)

def target_function(X1, X2, X3, df):
    df["CBTS"] += X1
    df["EPDS"] += X2
    df["HADS"] += X3
    #print(df["CBTS"],df["EPDS"],df["HADS"])
    X = df[["母亲年龄","教育程度","CBTS",  "EPDS","HADS"]]
    X = X - mean
    X= X / std
    X_poly = poly.fit_transform(X)
    y_pred = model.predict(X_poly)
    X_ = df[["母亲年龄", "婚姻状况", "教育程度", "妊娠时间", "分娩方式", "CBTS", "EPDS", "HADS"]]
    X_=X_-mean2
    X_=X_/std2
    X1_poly2 = poly.fit_transform(X_)
    y_pred_=rf_model.predict(X1_poly2)
    if abs(y_pred - 2)< 0.5 and y_pred_ > 75:
        SS = 0
    else:
        SS = 1000000
    return k1*abs(X1)+b1+k2*abs(X2)+b2+k3*abs(X3)+b3+SS
    
# 模拟退火法函数
def simulated_annealing():
    CBTS = 15
    EPDS = 22
    HADS = 18
    # 初始解
    X1 = 0
    X2 = 0
    X3 = 0
    current_solution = (X1, X2, X3)
    current_fitness = target_function(X1, X2, X3, df)
 
    # 最优解
    best_solution = current_solution
    best_fitness = current_fitness
 
    # 初始温度和终止温度
    initial_temperature = 100
    final_temperature = 1
 
    # 退火率
    cooling_rate = 0.90
 
    # 记录适应度随迭代次数的变化
    fitness_evolution = []
 
    # 模拟退火过程
    while initial_temperature > final_temperature:
        for _ in range(100):
            # 生成新解
            new_X1 = np.clip(np.random.randint(current_solution[0]-2, current_solution[0]+2), -2, 1)
            new_X2 = np.clip(np.random.randint(current_solution[1]-2, current_solution[1]+2), -2, 1)
            new_X3 = np.clip(np.random.randint(current_solution[2]-2, current_solution[2]+2), -2, 1)
            new_solution = (new_X1, new_X2, new_X3)
            new_fitness = target_function(new_X1, new_X2, new_X3, df)
 
            # 计算适应度差值
            delta_fitness = new_fitness - current_fitness
 
            # 判断是否接受新解
            if delta_fitness < 0 or np.random.rand() < np.exp(-delta_fitness / initial_temperature):
                current_solution = new_solution
                current_fitness = new_fitness
 
            # 更新最优解
                if current_fitness < best_fitness:
                    best_solution = current_solution
                    best_fitness = current_fitness
 
        # 记录适应度随迭代次数的变化
        fitness_evolution.append(best_fitness)
 
        # 降低温度
        initial_temperature *= cooling_rate
 
    return best_solution, best_fitness, fitness_evolution
# 运行模拟退火法
best_solution, best_fitness, fitness_evolution = simulated_annealing()
# 输出结果
print("最小函数值：", best_fitness)
print("最优解：", best_solution)

0    15
Name: CBTS, dtype: int64 0    22
Name: EPDS, dtype: int64 0    18
Name: HADS, dtype: int64
0    13
Name: CBTS, dtype: int64 0    20
Name: EPDS, dtype: int64 0    19
Name: HADS, dtype: int64
0    13
Name: CBTS, dtype: int64 0    21
Name: EPDS, dtype: int64 0    20
Name: HADS, dtype: int64
0    12
Name: CBTS, dtype: int64 0    20
Name: EPDS, dtype: int64 0    18
Name: HADS, dtype: int64
0    12
Name: CBTS, dtype: int64 0    20
Name: EPDS, dtype: int64 0    17
Name: HADS, dtype: int64
0    11
Name: CBTS, dtype: int64 0    18
Name: EPDS, dtype: int64 0    16
Name: HADS, dtype: int64
0    9
Name: CBTS, dtype: int64 0    18
Name: EPDS, dtype: int64 0    15
Name: HADS, dtype: int64
0    7
Name: CBTS, dtype: int64 0    19
Name: EPDS, dtype: int64 0    14
Name: HADS, dtype: int64
0    5
Name: CBTS, dtype: int64 0    18
Name: EPDS, dtype: int64 0    12
Name: HADS, dtype: int64
0    3
Name: CBTS, dtype: int64 0    16
Name: EPDS, dtype: int64 0    12
Name: HADS, dtype: int64
0    1
Name: C